In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

df = pd.read_csv('Pendapatan.csv')
df_test = pd.read_csv('Pendapatan_test.csv')

df = df.drop(['Keuntungan Kapital', 'Kerugian Capital'], axis=1)
df_test = df_test.drop(['Keuntungan Kapital', 'Kerugian Capital'], axis=1)

df['Kelas Pekerja'] = df['Kelas Pekerja'].replace('?', np.nan)
df['Pekerjaan'] = df['Pekerjaan'].replace('?', np.nan)
df.fillna(method='ffill')
df_test['Kelas Pekerja'] = df_test['Kelas Pekerja'].replace('?', np.nan)
df_test['Pekerjaan'] = df_test['Pekerjaan'].replace('?', np.nan)
df_test.fillna(method='ffill')

obj_pendidikan = {'1st-4th' : 0, '5th-6th' : 1, '7th-8th' : 2, '9th' : 3, '10th' : 4, '11th' : 5, '12th' : 6, 'SD' : 7, 'SMA' : 8, 'Pendidikan Tinggi' : 9, 'D3' : 10, 'D4' : 11, 'Sarjana' : 12, 'Master' : 13, 'Doktor' : 14, 'Sekolah Professional' : 15}
df['Pendidikan'] = df['Pendidikan'].replace(obj_pendidikan)
df = pd.get_dummies(df, columns=['Kelas Pekerja', 'Status Perkawinan', 'Pekerjaan', 'Jenis Kelamin'])
df_test['Pendidikan'] = df_test['Pendidikan'].replace(obj_pendidikan)
df_test = pd.get_dummies(df_test, columns=['Kelas Pekerja', 'Status Perkawinan', 'Pekerjaan', 'Jenis Kelamin'])

df_id = df['id']
df = df.drop('id', axis=1)
df_test_id = df_test['id'] 
df_test = df_test.drop('id', axis=1)

x = df.drop('Gaji', axis=1)
y = df['Gaji']

stdscalar = StandardScaler().fit(x)
x = pd.DataFrame(stdscalar.fit_transform(x), columns=x.columns)
df_test = pd.DataFrame(stdscalar.fit_transform(df_test), columns=df_test.columns)

# model = RandomForestClassifier(
#     n_estimators=1000, 
#     criterion='entropy', 
#     random_state=1000, 
#     class_weight='balanced'
# )
# model.fit(x, y)
# y_pred = model.predict(df_test)

# df_test['id'] = df_test_id
# df_test['Gaji'] = y_pred

# print(roc_auc_score(y, model.predict(x)))
# print(df_test['Gaji'].value_counts())

In [0]:
rfc = RandomForestClassifier() 

param_grid = { 
    'n_estimators': np.arange(100, 1000, 100),
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']
}

CV_rfc = RandomizedSearchCV(rfc, param_grid, cv= 10, scoring='roc_auc')
CV_rfc.fit(x, y)
print('Best Params :' , CV_rfc.best_params_)
print('Best Score :' , CV_rfc.best_score_)

In [68]:
gscv.best_params_

{'metric': 'manhattan', 'n_neighbors': 4, 'weights': 'distance'}

In [69]:
model = KNeighborsClassifier(metric='manhattan', n_neighbors=4, weights='distance')
model.fit(x,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='distance')

In [70]:
y_pred = model.predict(df_test)

df_test['id'] = df_test_id
df_test['Gaji'] = y_pred

print(roc_auc_score(y, model.predict(x)))
print(df_test['Gaji'].value_counts())

0.9997281131049484
0    7516
1    2083
Name: Gaji, dtype: int64


In [0]:
df_test.drop(df_test.columns.difference(['id','Gaji']), axis=1, inplace=True)
df_test.to_csv("Pendapatan_test_mazharrasyad-10.csv", index=False)

In [72]:
df_test.Gaji.value_counts()

0    7516
1    2083
Name: Gaji, dtype: int64